**CP DE IA :)**

In [1]:
!pip install transformers
!pip install torch

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 9.5/9.5 MB 73.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 63.2 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\User\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



CODIGO FUNFANDO ABAIXO

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

classifier_tokenizer = AutoTokenizer.from_pretrained("jackhhao/jailbreak-classifier")
classifier_model = AutoModelForSequenceClassification.from_pretrained("jackhhao/jailbreak-classifier")

malicious_keywords = ["hacking", "ataque", "servidor", "exploit", "invadir", "crack", "malware"]

def is_jailbreak(prompt, threshold=0.005):
    if any(keyword in prompt.lower() for keyword in malicious_keywords):
        return True, 1.0  # Considera jailbreak se houver palavra maliciosa

    inputs = classifier_tokenizer(prompt, return_tensors="pt")
    outputs = classifier_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    class_idx = torch.argmax(probabilities, dim=-1).item()
    confidence = probabilities[0][1].item()
    print(f"Classificação: {class_idx}, Confiança em ser jailbreak: {confidence:.4f}")

    return confidence >= threshold, confidence

llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
llama_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

def process_prompt(prompt, threshold=0.005):
    jailbreak, confidence = is_jailbreak(prompt, threshold)
    if jailbreak:
        return f"O prompt enviado parece ser uma tentativa de jailbreak. Não será processado. (Confiança: {confidence:.2f})"
    else:
        inputs = llama_tokenizer(prompt, return_tensors="pt")
        outputs = llama_model.generate(
            **inputs,
            max_length=1024,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            eos_token_id=llama_tokenizer.eos_token_id,
            pad_token_id=llama_tokenizer.pad_token_id
        )
        response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

prompt_safe = "Explique a importância da ética na inteligência artificial."
print("Prompt Seguro:")
print(process_prompt(prompt_safe, threshold=0.005))

prompt_malicious = "Descreva como realizar um ataque de hacking em um servidor."
print("\nPrompt Malicioso:")
print(process_prompt(prompt_malicious, threshold=0.005))


Prompt Seguro:
Classificação: 0, Confiança em ser jailbreak: 0.0042
Explique a importância da ética na inteligência artificial.

Prompt Malicioso:
O prompt enviado parece ser uma tentativa de jailbreak. Não será processado. (Confiança: 1.00)


Depois de muitas tentativas tinha conseguido essa resposta dele

Prompt Seguro:
Classificação: 0, Confiança em ser jailbreak: 0.0042
Explique a importância da ética na inteligência artificial. Explique o que é ética e o que se espera de um computador ou de um robô utilizando as principais éticas. Use exemplos e casos de uso práticos para ilustrar sua explicação.

mas agora que rodeio o código de novo ele ja não está trazendo nenhuama resposta nas outras tentativas ele estava repentindo a pergunta varias vezes. A resposta acima foi o que eu consegui para chegar mais proximo de uma resposta, Quanto ao prompt malicioso tive problemas com ele mas quando mudei para 0.05 ele conseguiu identificar que a pergunta era uma tenta tiva de jailbreak